In [18]:
import pandas as pd
import altair as alt
from vega_datasets import data as vega_data

In [19]:
data = pd.read_csv('cleaned_us_map_data.csv')
print(data.head())

      State            Ban_Type
0   Alabama  Total abortion ban
1  Arkansas  Total abortion ban
2     Idaho  Total abortion ban
3   Indiana  Total abortion ban
4  Kentucky  Total abortion ban


In [20]:
# Add a mapping of State names to FIPS codes
state_fips = {
    "Alabama": 1, "Alaska": 2, "Arizona": 4, "Arkansas": 5, "California": 6,
    "Colorado": 8, "Connecticut": 9, "Delaware": 10, "District of Columbia": 11,
    "Florida": 12, "Georgia": 13, "Hawaii": 15, "Idaho": 16, "Illinois": 17,
    "Indiana": 18, "Iowa": 19, "Kansas": 20, "Kentucky": 21, "Louisiana": 22,
    "Maine": 23, "Maryland": 24, "Massachusetts": 25, "Michigan": 26,
    "Minnesota": 27, "Mississippi": 28, "Missouri": 29, "Montana": 30,
    "Nebraska": 31, "Nevada": 32, "New Hampshire": 33, "New Jersey": 34,
    "New Mexico": 35, "New York": 36, "North Carolina": 37, "North Dakota": 38,
    "Ohio": 39, "Oklahoma": 40, "Oregon": 41, "Pennsylvania": 42,
    "Rhode Island": 44, "South Carolina": 45, "South Dakota": 46,
    "Tennessee": 47, "Texas": 48, "Utah": 49, "Vermont": 50, "Virginia": 51,
    "Washington": 53, "West Virginia": 54, "Wisconsin": 55, "Wyoming": 56
}

# Map State names to FIPS codes
data['id'] = data['State'].map(state_fips)

In [21]:
data = data[data['Ban_Type'].notnull()]

In [22]:
# Load the U.S. states map
us_states = alt.topo_feature(vega_data.us_10m.url, 'states')

In [23]:
# Create the U.S. map visualization
map_chart = alt.Chart(us_states).mark_geoshape(
    stroke='#aaa', strokeWidth=0.25
).transform_lookup(
    lookup='id',  # Match the state FIPS codes
    from_=alt.LookupData(data, key='id', fields=['State', 'Ban_Type'])
).encode(
    color=alt.Color('Ban_Type:N', scale=alt.Scale(scheme='category20b'), title="Abortion Policy"),
    tooltip=['State:N', 'Ban_Type:N']  # Display state and ban type on hover
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500,
    title="Abortion Policies Across U.S. States"
).configure_view(
    stroke=None
)

# Display the chart
map_chart.show()

alt.Chart(...)

WITH SELECTION

In [24]:
# Create a multi-select dropdown for Ban_Type
selection = alt.selection_multi(fields=['Ban_Type'], bind='legend')

# Create the U.S. map visualization
map_chart = alt.Chart(us_states).mark_geoshape(
    stroke='#aaa', strokeWidth=0.25
).transform_lookup(
    lookup='id',  # Match the state FIPS codes
    from_=alt.LookupData(data, key='id', fields=['State', 'Ban_Type'])
).encode(
    color=alt.condition(
        selection, 
        alt.Color('Ban_Type:N', scale=alt.Scale(scheme='category20b'), title="Abortion Policy"), 
        alt.value('lightgray')  # Non-selected states are grayed out
    ),
    tooltip=['State:N', 'Ban_Type:N']  # Display state and ban type on hover
).add_selection(
    selection
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500,
    title="Abortion Policies Across U.S. States"
).configure_view(
    stroke=None
)

# Display the chart
map_chart.show()


/var/folders/np/l7kf7rgn071d7c4hcsc3kc7w0000gn/T/ipykernel_60831/2211908841.py:2: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use selection_point instead.
  selection = alt.selection_multi(fields=['Ban_Type'], bind='legend')
/var/folders/np/l7kf7rgn071d7c4hcsc3kc7w0000gn/T/ipykernel_60831/2211908841.py:17: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  ).add_selection(


alt.Chart(...)

In [25]:
multi_select = alt.selection_multi(fields=['Ban_Type'], bind='legend')

# Create the U.S. map visualization
map_chart = alt.Chart(us_states).mark_geoshape(
    stroke='#aaa', strokeWidth=0.25
).transform_lookup(
    lookup='id',  # Match the state FIPS codes
    from_=alt.LookupData(data, key='id', fields=['State', 'Ban_Type'])
).encode(
    color=alt.condition(
        multi_select,
        alt.Color('Ban_Type:N', scale=alt.Scale(scheme='category20b'), title="Abortion Policy"),
        alt.value('lightgray')# Non-selected states are grayed out
    ),
    tooltip=['State:N', 'Ban_Type:N']  # Display state and ban type on hover
).add_selection(
    multi_select
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500,
    title=alt.TitleParams(
        text="Abortion Policies Across U.S. States",
        fontSize=20  # Make the title font larger
    )
).configure_legend(
    labelFontSize=14, # Make the legend text larger
    titleFontSize=16,   # Make the legend text larger
    symbolType='square',  # Change legend markers to squares
    symbolSize=200  # Make legend markers larger
).configure_view(
    stroke=None
)

# Display the chart
map_chart.show()

/var/folders/np/l7kf7rgn071d7c4hcsc3kc7w0000gn/T/ipykernel_60831/3805740611.py:1: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use selection_point instead.
  multi_select = alt.selection_multi(fields=['Ban_Type'], bind='legend')
/var/folders/np/l7kf7rgn071d7c4hcsc3kc7w0000gn/T/ipykernel_60831/3805740611.py:16: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  ).add_selection(


alt.Chart(...)

UPDATE

In [26]:
us_states = alt.topo_feature(vega_data.us_10m.url, 'states')

# Create a dropdown menu for Ban_Type
dropdown = alt.binding_select(options=list(data['Ban_Type'].unique()), name="Select Ban Type:")
selection = alt.selection_single(fields=['Ban_Type'], bind=dropdown)

# Create the U.S. map visualization
map_chart = alt.Chart(us_states).mark_geoshape(
    stroke='#aaa', strokeWidth=0.25
).transform_lookup(
    lookup='id',  # Match the state FIPS codes
    from_=alt.LookupData(data, key='id', fields=['State', 'Ban_Type'])
).encode(
    color=alt.condition(
        selection,
        alt.Color('Ban_Type:N', scale=alt.Scale(scheme='category20b'), title="Abortion Policy"),
        alt.value('lightgray')  # Non-selected states are grayed out
    ),
    tooltip=['State:N', 'Ban_Type:N']  # Display state and ban type on hover
).add_selection(
    selection
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500,
    title="Abortion Policies Across U.S. States"
).configure_view(
    stroke=None
)

# Display the chart
map_chart.show()

/var/folders/np/l7kf7rgn071d7c4hcsc3kc7w0000gn/T/ipykernel_60831/1501872413.py:5: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use selection_point instead.
  selection = alt.selection_single(fields=['Ban_Type'], bind=dropdown)
/var/folders/np/l7kf7rgn071d7c4hcsc3kc7w0000gn/T/ipykernel_60831/1501872413.py:20: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  ).add_selection(


alt.Chart(...)

In [42]:
import pandas as pd
import altair as alt
from vega_datasets import data as vega_data

# Load the dataset
data = pd.read_csv('cleaned_us_map_data.csv')

# Add a mapping of State names to FIPS codes
state_fips = {
    "Alabama": 1, "Alaska": 2, "Arizona": 4, "Arkansas": 5, "California": 6,
    "Colorado": 8, "Connecticut": 9, "Delaware": 10, "District of Columbia": 11,
    "Florida": 12, "Georgia": 13, "Hawaii": 15, "Idaho": 16, "Illinois": 17,
    "Indiana": 18, "Iowa": 19, "Kansas": 20, "Kentucky": 21, "Louisiana": 22,
    "Maine": 23, "Maryland": 24, "Massachusetts": 25, "Michigan": 26,
    "Minnesota": 27, "Mississippi": 28, "Missouri": 29, "Montana": 30,
    "Nebraska": 31, "Nevada": 32, "New Hampshire": 33, "New Jersey": 34,
    "New Mexico": 35, "New York": 36, "North Carolina": 37, "North Dakota": 38,
    "Ohio": 39, "Oklahoma": 40, "Oregon": 41, "Pennsylvania": 42,
    "Rhode Island": 44, "South Carolina": 45, "South Dakota": 46,
    "Tennessee": 47, "Texas": 48, "Utah": 49, "Vermont": 50, "Virginia": 51,
    "Washington": 53, "West Virginia": 54, "Wisconsin": 55, "Wyoming": 56
}

# Map State names to FIPS codes
data['id'] = data['State'].map(state_fips)

# Explicitly remove rows where Ban_Type is null
data = data.dropna(subset=['Ban_Type'])

# Load the U.S. states map
us_states = alt.topo_feature(vega_data.us_10m.url, 'states')

# Create a multi-select using `alt.selection_multi`
multi_select = alt.selection_multi(fields=['Ban_Type'], bind='legend')

# Create the U.S. map visualization (with fill)
filled_map = alt.Chart(us_states).mark_geoshape(
    stroke=None  # No border for the filled layer
).transform_lookup(
    lookup='id',  # Match the state FIPS codes
    from_=alt.LookupData(data, key='id', fields=['State', 'Ban_Type'])
).encode(
    color=alt.condition(
        multi_select,
        alt.Color('Ban_Type:N', scale=alt.Scale(scheme='purples'), title="Abortion Policy"),
        alt.value('lightgray')  # Non-selected states are grayed out
    ),
    tooltip=['State:N', 'Ban_Type:N']  # Display state and ban type on hover
).add_selection(
    multi_select
)

# Create the border layer
borders = alt.Chart(us_states).mark_geoshape(
    fillOpacity=0,  # Transparent fill
    stroke='black',  # Black borders
    strokeWidth=0.5  # Border thickness
).project(
    type='albersUsa'
)

# Combine the layers
map_chart = (filled_map + borders).properties(
    width=900,
    height=500,
    title=alt.TitleParams(
        text="Abortion Policies Across U.S. States",
        fontSize=20  # Make the title font larger
    )
).configure_legend(
    labelFontSize=14,  # Make the legend text larger
    titleFontSize=16,  # Make the legend title larger
    symbolType='square',  # Change legend markers to squares
    symbolSize=200  # Make legend markers larger
).configure_view(
    stroke=None
)

# Display the chart
map_chart.show()


/var/folders/np/l7kf7rgn071d7c4hcsc3kc7w0000gn/T/ipykernel_60831/2714099711.py:34: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use selection_point instead.
  multi_select = alt.selection_multi(fields=['Ban_Type'], bind='legend')
/var/folders/np/l7kf7rgn071d7c4hcsc3kc7w0000gn/T/ipykernel_60831/2714099711.py:49: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  ).add_selection(


alt.LayerChart(...)

In [28]:
print(data['Ban_Type'].unique())

['Total abortion ban' '6-week ban' '12-week ban' '15-week ban'
 '18-23-week ban' 'No gestational ban']


In [29]:
# Example of standardizing state names
data['State'] = data['State'].str.strip()


In [30]:
# Map FIPS codes to the dataset
data['FIPS'] = data['State'].map(state_fips)


In [31]:
data = data.dropna(subset=['Ban_Type', 'FIPS'])


In [32]:
missing_states = set(data['State'].unique()) - set(state_fips.keys())
print("States in dataset but not in FIPS mapping:", missing_states)

extra_states = set(state_fips.keys()) - set(data['State'].unique())
print("States in FIPS mapping but not in dataset:", extra_states)


States in dataset but not in FIPS mapping: set()
States in FIPS mapping but not in dataset: set()


In [1]:
import pandas as pd
import altair as alt
from vega_datasets import data as vega_data

# Load the dataset
data = pd.read_csv('cleaned_us_map_data.csv')

# Add a mapping of State names to FIPS codes
state_fips = {
    "Alabama": 1, "Alaska": 2, "Arizona": 4, "Arkansas": 5, "California": 6,
    "Colorado": 8, "Connecticut": 9, "Delaware": 10, "District of Columbia": 11,
    "Florida": 12, "Georgia": 13, "Hawaii": 15, "Idaho": 16, "Illinois": 17,
    "Indiana": 18, "Iowa": 19, "Kansas": 20, "Kentucky": 21, "Louisiana": 22,
    "Maine": 23, "Maryland": 24, "Massachusetts": 25, "Michigan": 26,
    "Minnesota": 27, "Mississippi": 28, "Missouri": 29, "Montana": 30,
    "Nebraska": 31, "Nevada": 32, "New Hampshire": 33, "New Jersey": 34,
    "New Mexico": 35, "New York": 36, "North Carolina": 37, "North Dakota": 38,
    "Ohio": 39, "Oklahoma": 40, "Oregon": 41, "Pennsylvania": 42,
    "Rhode Island": 44, "South Carolina": 45, "South Dakota": 46,
    "Tennessee": 47, "Texas": 48, "Utah": 49, "Vermont": 50, "Virginia": 51,
    "Washington": 53, "West Virginia": 54, "Wisconsin": 55, "Wyoming": 56
}

# Map State names to FIPS codes
data['id'] = data['State'].map(state_fips)

# Remove rows where Ban_Type is null
data['Ban_Type'] = data['Ban_Type'].astype(str)
data = data[data['Ban_Type'] != 'null']
data = data[data['Ban_Type'].notnull() & (data['Ban_Type'] != '')]

# Define a custom legend order
legend_order = [
    "Total abortion ban",
    "6-week ban",
    "12-week ban",
    "15-week ban",
    "18-23-week ban",
    "No gestational ban",
]

# Inspect the unique Ban_Type values
print("Unique Ban Types:", data['Ban_Type'].unique())

# Load the U.S. states map
us_states = alt.topo_feature(vega_data.us_10m.url, 'states')

# Create a multi-select using `alt.selection_multi`
multi_select = alt.selection_multi(fields=['Ban_Type'], bind='legend')

# Create the U.S. map visualization
filled_map = alt.Chart(us_states).mark_geoshape(
    stroke=None  # No border for the filled layer
).transform_lookup(
    lookup='id',  # Match the state FIPS codes
    from_=alt.LookupData(data, key='id', fields=['State', 'Ban_Type'])
).encode(
    color=alt.condition(
        multi_select,
        alt.Color(
            'Ban_Type:N',
            scale=alt.Scale(
                domain=[
                    "Total abortion ban",
                    "6-week ban",
                    "12-week ban",
                    "15-week ban",
                    "18-23-week ban",
                    "No gestational ban"
                ],
                scheme='bluepurple'
            ),
            title="Abortion Policy"
        ),
        alt.value('lightgray')  # Color for non-selected states
    ),
    tooltip=['State:N', 'Ban_Type:N']  # Display state and ban type on hover
).add_selection(
    multi_select
)

# Create the border layer
borders = alt.Chart(us_states).mark_geoshape(
    fillOpacity=0,  # Transparent fill
    stroke='black',  # Black borders
    strokeWidth=0.5  # Border thickness
).project(
    type='albersUsa'
)

# Combine the layers
map_chart = (filled_map + borders).properties(
    width=900,
    height=500,
    title=alt.TitleParams(
        text="Abortion Policies Across U.S. States",
        fontSize=20,
        font='Times New Roman'  # Make the title font larger
    )
).configure_legend(
    labelFontSize=14,  
    titleFontSize=16, 
    symbolType='square',  
    symbolSize=200,
    labelFont='Times New Roman',  # Set legend label font
    titleFont='Times New Roman'   # Set legend title font
).configure_view(
    stroke=None
)

# Display the chart
map_chart.show()


Unique Ban Types: ['Total abortion ban' '6-week ban' '12-week ban' '15-week ban'
 '18-23-week ban' 'No gestational ban']


/var/folders/np/l7kf7rgn071d7c4hcsc3kc7w0000gn/T/ipykernel_72719/1489462955.py:49: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use selection_point instead.
  multi_select = alt.selection_multi(fields=['Ban_Type'], bind='legend')
/var/folders/np/l7kf7rgn071d7c4hcsc3kc7w0000gn/T/ipykernel_72719/1489462955.py:78: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  ).add_selection(


alt.LayerChart(...)

In [2]:
map_chart.save('abortion_policies_map.html')

In [ ]:
# import pandas as pd
# import altair as alt
# from vega_datasets import data as vega_data

# # Load the dataset
# data = pd.read_csv('cleaned_us_map_data.csv')

# # Add a mapping of State names to FIPS codes
# state_fips = {
#     "Alabama": 1, "Alaska": 2, "Arizona": 4, "Arkansas": 5, "California": 6,
#     "Colorado": 8, "Connecticut": 9, "Delaware": 10, "District of Columbia": 11,
#     "Florida": 12, "Georgia": 13, "Hawaii": 15, "Idaho": 16, "Illinois": 17,
#     "Indiana": 18, "Iowa": 19, "Kansas": 20, "Kentucky": 21, "Louisiana": 22,
#     "Maine": 23, "Maryland": 24, "Massachusetts": 25, "Michigan": 26,
#     "Minnesota": 27, "Mississippi": 28, "Missouri": 29, "Montana": 30,
#     "Nebraska": 31, "Nevada": 32, "New Hampshire": 33, "New Jersey": 34,
#     "New Mexico": 35, "New York": 36, "North Carolina": 37, "North Dakota": 38,
#     "Ohio": 39, "Oklahoma": 40, "Oregon": 41, "Pennsylvania": 42,
#     "Rhode Island": 44, "South Carolina": 45, "South Dakota": 46,
#     "Tennessee": 47, "Texas": 48, "Utah": 49, "Vermont": 50, "Virginia": 51,
#     "Washington": 53, "West Virginia": 54, "Wisconsin": 55, "Wyoming": 56
# }

# # Ensure District of Columbia is included in the dataset
# if "District of Columbia" not in data['State'].values:
#     data = pd.concat([
#         data,
#         pd.DataFrame({"State": ["District of Columbia"], "Ban_Type": ["No gestational ban"]})
#     ], ignore_index=True)

# # Map State names to FIPS codes
# data['id'] = data['State'].map(state_fips)

# # Remove rows where Ban_Type is null
# data['Ban_Type'] = data['Ban_Type'].astype(str)
# data = data[data['Ban_Type'] != 'null']
# data = data[data['Ban_Type'].notnull() & (data['Ban_Type'] != '')]

# # Define a custom legend order
# legend_order = [
#     "Total abortion ban",
#     "6-week ban",
#     "12-week ban",
#     "15-week ban",
#     "18-23-week ban",
#     "No gestational ban",
# ]

# # Inspect the unique Ban_Type values
# print("Unique Ban Types:", data['Ban_Type'].unique())

# # Load the U.S. states map
# us_states = alt.topo_feature(vega_data.us_10m.url, 'states')

# # Create a multi-select using `alt.selection_multi`
# multi_select = alt.selection_multi(fields=['Ban_Type'], bind='legend')

# # Create the U.S. map visualization
# filled_map = alt.Chart(us_states).mark_geoshape(
#     stroke=None  # No border for the filled layer
# ).transform_lookup(
#     lookup='id',  # Match the state FIPS codes
#     from_=alt.LookupData(data, key='id', fields=['State', 'Ban_Type'])
# ).encode(
#     color=alt.condition(
#         multi_select,
#         alt.Color(
#             'Ban_Type:N',
#             scale=alt.Scale(
#                 domain=[
#                     "Total abortion ban",
#                     "6-week ban",
#                     "12-week ban",
#                     "15-week ban",
#                     "18-23-week ban",
#                     "No gestational ban"
#                 ],
#                 scheme='purples'
#             ),
#             title="Abortion Policy"
#         ),
#         alt.value('lightgray')  # Color for non-selected states
#     ),
#     tooltip=['State:N', 'Ban_Type:N']  # Display state and ban type on hover
# ).add_selection(
#     multi_select
# )

# # Combine the layers
# map_chart = (filled_map + borders).properties(
#     width=700,
#     height=500,
#     title=alt.TitleParams(
#         text="Abortion Policies Across U.S. States",
#         fontSize=20  # Make the title font larger
#     )
# ).configure_legend(
#     labelFontSize=16,  # Increase font size for legend labels
#     titleFontSize=20,  # Increase font size for the legend title
#     symbolType='square',  # Use squares for legend symbols
#     symbolSize=300  # Increase the size of legend symbols
# ).configure_view(
#     stroke=None  # Remove stroke from the view
# ).configure(
#     background='#F5F5F5'  # Set the overall chart background color
# )



# # Display the chart
# map_chart.show()


Unique Ban Types: ['Total abortion ban' '6-week ban' '12-week ban' '15-week ban'
 '18-23-week ban' 'No gestational ban']


/var/folders/np/l7kf7rgn071d7c4hcsc3kc7w0000gn/T/ipykernel_60831/1754030996.py:56: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use selection_point instead.
  multi_select = alt.selection_multi(fields=['Ban_Type'], bind='legend')
/var/folders/np/l7kf7rgn071d7c4hcsc3kc7w0000gn/T/ipykernel_60831/1754030996.py:85: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  ).add_selection(


alt.LayerChart(...)

In [ ]:
# map_chart.save('abortion_policies_map.html')

In [37]:
# Inspect all unique values in Ban_Type to confirm if 'null' exists
print(data['Ban_Type'].unique())


['Total abortion ban' '6-week ban' '12-week ban' '15-week ban'
 '18-23-week ban' 'No gestational ban']
